In [1]:
class Operation():
    def __init__(self,input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        for nodes in input_nodes:
            
            nodes.output_nodes.append(self)
        _default_graph.operation.append(self)   
    def compute(self):
        pass
            

In [2]:
class Add(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var+y_var

In [4]:
class Multiply(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var*y_var

In [5]:
class Matmul(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var.dot(y_var)

In [6]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [7]:
class Variable():
    def __init__(self,initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        _default_graph.variables.append(self)

In [8]:
class Graph():
    def __init__(self):
        self.placeholders = []
        self.variables = []
        self.operation = []
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [9]:
#creating like z = Ax+b
g = Graph()
g.set_as_default()
A = Variable(10)
b = Variable(1)
x = Placeholder()
y = Multiply(A,x)
z= Add(y,b)

In [10]:
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node,Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder

In [11]:
class Session():
    
    
    def run(self,operation,feed_dict= {}):
        nodes_postorder = traverse_postorder(operation)
        for node in nodes_postorder:
            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
            if type(node.output)==list:
                node.output = np.array(node.output)
        return operation.output

In [12]:
sess = Session()

In [13]:
result = sess.run(operation=z,feed_dict={x:10})

In [14]:
result

101